# 🖥️ 06: Filesystem and Code Execution

Learn how to give your LLM the ability to execute Python code and perform file operations, enabling it to compute results and persist data.

## 📋 Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Execute Python code dynamically using the `execute_python` tool
- [ ] Perform file operations with the `filesystem_operation` tool
- [ ] Understand working directory concepts for file paths
- [ ] Separate concerns: computation vs. I/O operations
- [ ] Handle security considerations for code execution
- [ ] Build a file-based counter using both tools
- [ ] Understand when to use each tool appropriately

## 🎯 Prerequisites

- Completed notebook 05 (Custom Tools)
- Understanding of tool registration and usage
- Basic Python file operations knowledge
- Awareness of code execution security implications

## ⏱️ Estimated Time: 15 minutes

## ⚠️ Security Warning

**IMPORTANT:** The `execute_python` tool executes arbitrary Python code. This is powerful but potentially dangerous:

- Only use in trusted environments
- Never expose to untrusted users without sandboxing
- Be aware that code can modify files, network, etc.
- Consider using restricted execution environments for production

For learning purposes in this notebook, we're using it in a controlled manner.

## 1️⃣ The execute_python Tool

The `execute_python` tool runs Python code dynamically and returns the results.

In [ ]:
from local_llm_sdk import LocalLLMClient
from local_llm_sdk.tools import get_builtin_tools

# Create client and register all built-in tools
client = LocalLLMClient(
    base_url="http://169.254.83.107:1234/v1",
    model="your-model-name"
)

tools = get_builtin_tools()
client.register_tools(tools)

print("✅ Registered tools:")
for tool_name in client.tools.list_tools():
    print(f"  - {tool_name}")

Let's execute some Python code:

In [ ]:
# Simple calculation
response = client.chat("Use Python to calculate the factorial of 10")
print("Factorial calculation:")
print(response)

**💡 What happened:**
1. LLM recognized it needs to execute Python code
2. It called `execute_python` with code like: `import math; math.factorial(10)`
3. The code was executed in a Python interpreter
4. The result was returned to the LLM
5. The LLM formatted a response

## 2️⃣ More Complex Computations

The execute_python tool can handle complex calculations and algorithms.

In [ ]:
# Statistical analysis
response = client.chat(
    "Calculate the mean, median, and standard deviation of these numbers: "
    "[23, 45, 12, 67, 89, 34, 56, 78, 90, 21]"
)
print("Statistics:")
print(response)

print("\n" + "="*70 + "\n")

# Generate sequence
response = client.chat(
    "Generate the first 15 Fibonacci numbers using Python"
)
print("Fibonacci sequence:")
print(response)

print("\n" + "="*70 + "\n")

# String manipulation
response = client.chat(
    "Use Python to find all palindromes in this list: "
    "['hello', 'racecar', 'python', 'level', 'world', 'madam']"
)
print("Palindrome finder:")
print(response)

## 3️⃣ The filesystem_operation Tool

The `filesystem_operation` tool allows reading, writing, and listing files.

In [ ]:
import os
import tempfile

# Create a temporary directory for our examples
temp_dir = tempfile.mkdtemp()
print(f"📁 Working directory: {temp_dir}\n")

# Write a file
response = client.chat(
    f"Create a file called 'hello.txt' in {temp_dir} with the content: 'Hello from the LLM!'"
)
print("File creation:")
print(response)

Read the file back:

In [ ]:
# Read the file
response = client.chat(f"Read the contents of {temp_dir}/hello.txt")
print("File contents:")
print(response)

List files in directory:

In [ ]:
# List directory
response = client.chat(f"List all files in {temp_dir}")
print("Directory listing:")
print(response)

**💡 filesystem_operation supports:**
- `read`: Read file contents
- `write`: Create or overwrite files
- `list`: List directory contents
- `append`: Add to existing files (if supported)
- `delete`: Remove files (if supported)

## 4️⃣ Combining Computation and File I/O

Let's combine `execute_python` (for computation) and `filesystem_operation` (for storage).

In [ ]:
# Create a data processing pipeline
response = client.chat(
    f"Generate a list of 20 random numbers between 1 and 100 using Python, "
    f"calculate their mean and median, "
    f"then save the numbers and statistics to {temp_dir}/data_analysis.txt"
)

print("Data processing pipeline:")
print(response)

Verify the output file:

In [ ]:
# Read the analysis file
response = client.chat(f"Read {temp_dir}/data_analysis.txt")
print("Analysis results:")
print(response)

## 5️⃣ Tool Separation: When to Use Each

Understanding when to use `execute_python` vs `filesystem_operation`:

### execute_python: For Computation

Use when you need to:
- Perform calculations
- Run algorithms
- Process data structures
- Use Python libraries
- Generate sequences or patterns

**Example tasks:**
- Sort an array
- Calculate statistics
- Parse dates
- Generate random data
- Apply mathematical formulas

In [ ]:
# Good use of execute_python
response = client.chat(
    "Sort these numbers in descending order and show every 3rd element: "
    "[45, 12, 78, 23, 89, 34, 56, 67, 90, 21, 43, 65, 87, 32, 54]"
)
print("Computation example:")
print(response)

### filesystem_operation: For I/O

Use when you need to:
- Read data from files
- Save results to disk
- List directory contents
- Persist state between conversations
- Load configuration

**Example tasks:**
- Save report to file
- Read CSV data
- Create log entries
- Store user preferences
- Load templates

In [ ]:
# Good use of filesystem_operation
response = client.chat(
    f"Create a todo list file at {temp_dir}/todos.txt with these items:\n"
    "1. Learn about filesystem tools\n"
    "2. Practice code execution\n"
    "3. Build something cool"
)
print("File I/O example:")
print(response)

## 6️⃣ Working Directory Considerations

File paths can be relative or absolute. Understanding the working directory is important.

In [ ]:
# Check current working directory
response = client.chat("Use Python to print the current working directory")
print("Current working directory:")
print(response)

print("\n💡 Tip: Always use absolute paths when working with files to avoid confusion!")
print(f"   Example absolute path: {temp_dir}/myfile.txt")
print(f"   Example relative path: myfile.txt (depends on CWD)")

## 🏋️ Exercise: File-Based Counter

**Challenge:** Build a persistent counter that:
1. Stores its value in a file called `counter.txt`
2. Reads the current value from the file (defaults to 0 if file doesn't exist)
3. Increments the counter by a specified amount
4. Saves the new value back to the file
5. Returns the old value, new value, and increment amount

Requirements:
- Use `filesystem_operation` to read and write the file
- Use `execute_python` if needed for calculations
- Handle the case where the file doesn't exist yet
- Test incrementing multiple times

Try it yourself first!

In [ ]:
# Your code here:



<details>
<summary>Click to see solution</summary>

```python
# Solution: File-based persistent counter

import os

counter_file = os.path.join(temp_dir, "counter.txt")

print("🔢 File-Based Counter Application\n")
print("="*70 + "\n")
print(f"Counter file: {counter_file}\n")

# Initialize counter (first use)
print("Step 1: Initialize counter")
response = client.chat(
    f"Check if {counter_file} exists. If not, create it with value '0'. "
    f"If it exists, read its current value."
)
print(response)
print()

# Increment by 5
print("Step 2: Increment by 5")
response = client.chat(
    f"Read the number from {counter_file}, add 5 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Increment by 10
print("Step 3: Increment by 10")
response = client.chat(
    f"Read the number from {counter_file}, add 10 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Increment by 3
print("Step 4: Increment by 3")
response = client.chat(
    f"Read the number from {counter_file}, add 3 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Final check
print("Step 5: Verify final value")
response = client.chat(f"What is the current value in {counter_file}?")
print(response)
print()

print("="*70)
print("\n✅ Counter successfully persisted across multiple operations!")
print(f"   Expected final value: 18 (0 + 5 + 10 + 3)")
```
</details>

In [ ]:
# Solution cell (run this to see the answer)
counter_file = os.path.join(temp_dir, "counter.txt")

print("🔢 File-Based Counter Application\n")
print("="*70 + "\n")
print(f"Counter file: {counter_file}\n")

# Initialize
print("Step 1: Initialize counter")
response = client.chat(
    f"Check if {counter_file} exists. If not, create it with value '0'. "
    f"If it exists, read its current value."
)
print(response)
print()

# Increment by 5
print("Step 2: Increment by 5")
response = client.chat(
    f"Read the number from {counter_file}, add 5 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Increment by 10
print("Step 3: Increment by 10")
response = client.chat(
    f"Read the number from {counter_file}, add 10 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Increment by 3
print("Step 4: Increment by 3")
response = client.chat(
    f"Read the number from {counter_file}, add 3 to it, "
    f"save the new value back to the file, and tell me both the old and new values."
)
print(response)
print()

# Final check
print("Step 5: Verify final value")
response = client.chat(f"What is the current value in {counter_file}?")
print(response)
print()

print("="*70)
print("\n✅ Counter successfully persisted across multiple operations!")
print(f"   Expected final value: 18 (0 + 5 + 10 + 3)")

## 🧹 Cleanup

Let's clean up our temporary directory:

In [ ]:
import shutil

# Remove temporary directory
shutil.rmtree(temp_dir)
print(f"✅ Cleaned up temporary directory: {temp_dir}")

## ⚠️ Common Pitfalls

### 1. Executing Unsafe Code
```python
# ❌ DANGEROUS: Never execute untrusted code
user_code = input("Enter code: ")  # User could enter malicious code!
response = client.chat(f"Execute this Python: {user_code}")

# ✅ Good: Validate and sanitize, or use sandboxing
# In production, use restricted execution environments
```

### 2. Using Relative Paths
```python
# ⚠️ Warning: Relative paths depend on working directory
response = client.chat("Read myfile.txt")  # Where is myfile.txt?

# ✅ Good: Use absolute paths
import os
file_path = os.path.abspath("myfile.txt")
response = client.chat(f"Read {file_path}")
```

### 3. Not Handling File Errors
```python
# ❌ Bad: Assuming file exists
response = client.chat(f"Read {nonexistent_file}")
# Will error if file doesn't exist

# ✅ Good: Check existence first or handle errors
response = client.chat(
    f"Check if {file_path} exists. If yes, read it. If no, say file not found."
)
```

### 4. Forgetting About Permissions
```python
# ⚠️ Warning: May not have write permissions
response = client.chat("Write to /root/protected.txt")
# Could fail due to permissions

# ✅ Good: Use directories you own
import tempfile
temp_dir = tempfile.mkdtemp()
response = client.chat(f"Write to {temp_dir}/file.txt")
```

### 5. Mixing Concerns
```python
# ⚠️ Suboptimal: Using execute_python for file I/O
response = client.chat(
    "Use Python to open 'data.txt', read it, and print contents"
)
# Works, but filesystem_operation is designed for this

# ✅ Better: Use the right tool for the job
response = client.chat("Read the file 'data.txt'")
# filesystem_operation handles this directly
```

## 🎓 What You Learned

✅ **execute_python**: Run Python code dynamically for computations

✅ **filesystem_operation**: Read, write, list files for I/O operations

✅ **Tool Separation**: Use execute_python for compute, filesystem_operation for I/O

✅ **Working Directories**: Understand absolute vs relative paths

✅ **Combining Tools**: Chain computation and storage for complete workflows

✅ **Security**: Be aware of code execution risks

✅ **Error Handling**: Check file existence, handle permissions

✅ **Best Practices**: Use absolute paths, validate inputs, choose appropriate tools

## 🎉 Congratulations!

You've completed the core tutorial series for the Local LLM SDK!

### What You've Mastered:

1. **Basic Chat**: Simple interactions, system prompts, temperature control
2. **Conversation History**: Multi-turn conversations with context
3. **Built-in Tools**: Using pre-made tools (calculator, text transformer, etc.)
4. **Custom Tools**: Creating your own tools with the @tool decorator
5. **Filesystem & Code**: Executing Python and managing files

### 🚀 What's Next?

Now that you have the foundations, you can:

**Build Real Applications:**
- Create a chatbot with tool capabilities
- Build a data analysis assistant
- Make an automation tool for repetitive tasks
- Develop a coding helper that executes and tests code

**Explore Advanced Topics:**
- Streaming responses for real-time output
- Error handling and retry logic
- Tool chaining for complex workflows
- Integration with external APIs and services
- Production deployment and security

**Check Out the SDK Documentation:**
- [README.md](../README.md) - Project overview and quick start
- [local_llm_sdk/](../local_llm_sdk/) - Source code with detailed docstrings
- `.documentation/` - Research and API comparisons

### 💬 Questions or Issues?

If you encounter any problems or have questions:
1. Check the SDK source code for implementation details
2. Review the documentation in `.documentation/`
3. Test with LM Studio server logs for debugging
4. Experiment in these notebooks to understand behavior

### 🌟 Keep Learning!

The best way to master the SDK is to build something. Start small, experiment often, and gradually increase complexity.

Happy coding with Local LLM SDK!